In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
!pip install -q datasist
from datasist.structdata import detect_outliers

In [ ]:
df=pd.read_csv("/kaggle/input/dubizzle-used-car-sale-data/data.csv",na_values=['NoneUnknown','Unknown','NA','inf'])
df.head()

In [ ]:
high_variance_ubpalanced_col=['mechanical_condition','mechanical_condition','date_posted','steering_side','motors_trim']

In [ ]:
df.drop('title',inplace=True,axis=1)

In [ ]:
for col in high_variance_ubpalanced_col:
    sns.countplot(data=df,y=df[col])
    plt.figure()

In [ ]:
df.drop(high_variance_ubpalanced_col,inplace=True,axis=1)

In [ ]:
#df.drop(['title','body_condition','date_posted'],inplace=True,axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
#df.dropna(inplace=True)

In [ ]:
df.info()

In [ ]:
def Price_in_aed_To_Int(price):
    num=""
    price_List=price.split(',')
    for i in price_List:
        num+=i
    return int(num)

df['price_in_aed']=df['price_in_aed'].apply(Price_in_aed_To_Int)

In [ ]:
sns.kdeplot(df['kilometers'])

In [ ]:
df.drop(df[df['kilometers']>300000].index,inplace=True,axis=0)

In [ ]:
sns.kdeplot(df['kilometers'])

In [ ]:
df['year'].astype(int,errors='ignore')

In [ ]:
df['no_of_cylinders'].unique()

In [ ]:
df.drop(df[df['no_of_cylinders']=='None'].index,axis=0,inplace=True)

In [ ]:
df['no_of_cylinders'].astype(int,errors='ignore')

In [ ]:
sns.countplot(data =df,y=df['company'])

In [ ]:
df['company'].value_counts().sort_values().head(29)

other_companies=df['company'].value_counts().sort_values().head(29).index
other_companies=list(other_companies)

In [ ]:
print(other_companies)

In [ ]:
def Handel_Companies(company):
    for c in range(len(other_companies)):
        if company==other_companies[c]:
            return 'other'
    return company

In [ ]:
df['company']=df['company'].apply(Handel_Companies)

sns.countplot(data =df,y=df['company'])

In [ ]:
sns.countplot(data =df,y=df['horsepower'])

In [ ]:
def Horsepower (horsepower):
    if horsepower == '700 - 800 HP' or horsepower == '800 - 900 HP' or horsepower == '900+ HP':
        return 'More than 700'
    else:
        return horsepower

df['horsepower']=df['horsepower'].apply(Horsepower)

In [ ]:
sns.countplot(data =df,y=df['horsepower'])

In [ ]:
sns.countplot(data=df,x='no_of_cylinders')

In [ ]:
not_important_no_of_cylinders_idx=df[(df['no_of_cylinders']=='3') | (df['no_of_cylinders']=='10') | (df['no_of_cylinders']=='5') | (df['no_of_cylinders']=='None')].index

In [ ]:
df.drop(not_important_no_of_cylinders_idx,axis=0,inplace=True)

In [ ]:
sns.countplot(data=df,x='no_of_cylinders')

In [ ]:
sns.countplot(y=df['year'])

In [ ]:
df.drop(df[df['year']<=2005].index,inplace=True,axis=0)

In [ ]:
sns.countplot(y=df['year'])

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    sns.countplot(data=df,y=df[col])
    plt.figure()

In [ ]:
print(list(df['model'].value_counts().sort_values().head(100)))

In [ ]:
other_model=list(df['model'].value_counts().sort_values().head(150).index)
print(other_model)

In [ ]:
def Handel_Model(model):
    for c in range(len(other_model)):
        if model==other_model[c]:
            return 'other'
    return model

In [ ]:
x=df['model'].apply(Handel_Model)

In [ ]:
sns.countplot(y=x)

In [ ]:
df['model']=x

In [ ]:
sns.displot(data=df,x=df['price_in_aed'])

In [ ]:
df.drop(df[df['price_in_aed']>1500000].index,inplace=True,axis=0)

In [ ]:
sns.displot(data=df,x=df['price_in_aed'])

In [ ]:
df['price_in_aed']=np.log(df['price_in_aed'])

In [ ]:
sns.displot(df['price_in_aed'])

In [ ]:
df.info()

In [ ]:
#df.drop('emirate',inplace=True,axis=1)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder , StandardScaler , LabelEncoder,MinMaxScaler
from category_encoders import BinaryEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
imputer=KNNImputer()


In [ ]:
x = df.drop("price_in_aed" , axis = 1)
y = df["price_in_aed"]

In [ ]:
df_catt=x.select_dtypes(include='object_')
df_catt


In [ ]:
df_num=x.select_dtypes(include=np.number)
df_num

In [ ]:
df.head(1)

In [ ]:
df['kilometers'].unique()

In [ ]:
Cat_Steps = list()
Cat_Steps.append(("imputer" , SimpleImputer(strategy="most_frequent")))
#Cat_Steps.append(("Encoder" , OneHotEncoder(sparse=False , drop="first")))
Cat_Steps.append(("Encoder" , BinaryEncoder()))
Categorical_pipeline = Pipeline(steps = Cat_Steps)

In [ ]:
Num_Steps = list()
Num_Steps.append(("imputer" , KNNImputer()))
Num_Steps.append(("scaler" , StandardScaler()))
Num_pipeline = Pipeline(steps = Num_Steps)

In [ ]:
Transform = ColumnTransformer(transformers=[("cat",Categorical_pipeline,df_catt.columns) , ("Num",Num_pipeline,df_num.columns)] ,remainder="passthrough")
Transform

In [ ]:
final_steps = list()
final_steps.append(("columns",Transform))
#final_steps.append(("scaler" , StandardScaler()))
#final_steps.append(("poly" , PolynomialFeatures(degree=2)))
#final_steps.append(("Model" , LinearRegression()))
final_pipeline = Pipeline(steps = final_steps)

final_pipeline


# Model selection


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense
from xgboost.sklearn import XGBRegressor
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
def create_model(optimizer='sgd',
                 kernel_initializer='normal'):
    model = Sequential()
    model.add(Dense(12,activation='relu'))
    model.add(Dense(15,activation='relu'))
    model.add(Dense(1,activation='relu',kernel_initializer='normal'))

    model.compile(loss='mean_squared_error',optimizer='sgd', metrics=['mse'])

    return model

In [ ]:
ANN = KerasRegressor(build_fn=create_model,verbose=0)

In [ ]:
models = list()
models.append(("LR" , LinearRegression()))
models.append(("RandomForestRegressor" , RandomForestRegressor()))
models.append(('GradientBoostingRegressor',GradientBoostingRegressor()))
models.append(("XGBRegressor",XGBRegressor()))
models.append(("SVM" , SVR()))
models.append(('ANN',ANN))

In [ ]:
for model in models:
    final_steps = list()
    final_steps.append(("columns",Transform))
    #final_steps.append(("scaler" , StandardScaler()))
    #final_steps.append(("poly" , PolynomialFeatures(degree=2)))

    final_steps.append(model)
    final_pipeline = Pipeline(steps = final_steps)

    scores = cross_validate(final_pipeline , x , y , cv = 5 , scoring='r2' , return_train_score=True)    
    print(model[0])
    print("Train_score" , scores["train_score"].mean() )
    print("-" * 10)
    print("Test_score" , scores["test_score"].mean())
    print("-" * 20)
    print("\n")
    

In [ ]:
from sklearn import metrics


In [ ]:
final_steps = list()
final_steps.append(("columns",Transform))
final_steps.append(("XGBRegressor",XGBRegressor()))
final_pipeline = Pipeline(steps = final_steps)

In [ ]:
scores = cross_validate(final_pipeline , x , y , cv = 5 , scoring=('r2', 'neg_mean_squared_error','neg_mean_absolute_error') , return_train_score=True)

In [ ]:
scores

In [ ]:
print("train_r2 =",scores["train_r2"].mean())
print('-'*50)
print("test_r2 =",scores["test_r2"].mean())
print('-'*50)
print("test_neg_mean_absolute_error =",abs(scores["test_neg_mean_absolute_error"].mean()))
print('-'*50)
print("test_neg_mean_squared_error =",abs(scores["test_neg_mean_squared_error"].mean()))

# Tuning

In [ ]:
params = [
    
    {   "XGBRegressor__max_depth" : range(5,10) ,
        "XGBRegressor__n_estimators":range(50,200,50),
        "XGBRegressor__learning_rate":[0.04,0.05,0.06,0.07,0.08,0.09]
    }
    
]

In [ ]:
grid = GridSearchCV(estimator = final_pipeline , param_grid = params , cv = 5  ,return_train_score=True ,  scoring = "r2" )

In [ ]:
grid.fit(x,y)

In [ ]:
grid.best_params_

In [ ]:
grid.cv_results_["mean_train_score"].max()

In [ ]:
grid.cv_results_["mean_test_score"].max()

# XGBRegressor Tuning

In [ ]:
final_steps = list()
final_steps.append(("columns",Transform))
final_steps.append(("XGBRegressor",XGBRegressor(learning_rate=0.09,max_depth=7,n_estimators=150)))
final_pipeline = Pipeline(steps = final_steps)

In [ ]:
scores = cross_validate(final_pipeline , x , y , cv = 5 , scoring=('r2', 'neg_mean_squared_error','neg_mean_absolute_error') , return_train_score=True)

In [ ]:
scores

In [ ]:
print("train_r2 =",scores["train_r2"].mean())
print('-'*50)
print("test_r2 =",scores["test_r2"].mean())
print('-'*50)
print("test_neg_mean_absolute_error =",abs(scores["test_neg_mean_absolute_error"].mean()))
print('-'*50)
print("test_neg_mean_squared_error =",abs(scores["test_neg_mean_squared_error"].mean()))